# Scraping de dados para a matrícula da UTFPR

### Carrega as bibliotecas & funções

In [2]:
from datetime import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup
from unidecode import unidecode
import re
import psycopg2
import yaml
from yaml.loader import SafeLoader

In [16]:
def find_table(soup: BeautifulSoup, pattern: str) -> str:
    h3_tags = soup.find_all('h3')
    for h3 in h3_tags:
        tag = h3.find(string=re.compile(f'^{pattern}$', re.I))
        if tag:
            return str(tag.find_next('table'))
    return None

def process_table(html_table: str, campus_id: pd.DataFrame, enrollment_phase_id: int, year: int, semester: int):
    if html_table:
        table = pd.read_html(html_table)[0]
        processed = pd.DataFrame()
        processed['campus'] = table.iloc[:, 0].apply(lambda x: unidecode(x).upper())
        processed['s_date_time'] = table.iloc[:, 1] + ' ' + table.iloc[:, 2]
        processed['start_timestamp'] = processed['s_date_time'].apply(lambda x: str(datetime.strptime(str(x).strip(), '%d/%m/%Y %Hh')))
        processed['e_date_time'] = table.iloc[:, 3] + ' ' + table.iloc[:, 4]
        processed['end_timestamp'] = processed['e_date_time'].apply(lambda x: str(datetime.strptime(str(x).strip(), '%d/%m/%Y %Hh')))
        processed['enrollment_phase_id'] = enrollment_phase_id
        processed['year'] = year
        processed['semester'] = semester
        processed = processed.join(campus_id.set_index('campus_name'), on='campus')
        return processed[['campus_id', 'enrollment_phase_id', 'year', 'semester', 'start_timestamp', 'end_timestamp']]
    return None

def psql_connect(credentials: str):
    """
    Establishes a connection to postgres
    """
    try:
        # read credentials
        with open(f'{credentials}', encoding='utf-8') as file:
            endpoints = yaml.load(file, Loader=SafeLoader)

        # parse credentials
        credentials = endpoints['tracker_store']
        host = credentials['url']
        database = credentials['db']
        user = credentials['username']
        password = credentials['password']

        try:
            conn = psycopg2.connect(
                host=host,
                database=database,
                user=user,
                password=password
            )

            return conn
        except psycopg2.OperationalError:
            return None

    except OSError:
        return None

### Carrega tabelas com códigos de campus & fase da matrícula do banco de dados

In [4]:
q_campus_id =\
"""
    SELECT CAMPUS_ID,
        CAMPUS_NAME
    FROM CAMPUS
    ORDER BY CAMPUS_ID ASC
"""
with psql_connect('../bot/endpoints.yml') as conn:
    campus_id = pd.read_sql_query(q_campus_id, conn)

In [5]:
campus_id

,campus_id,campus_name
0,1,CURITIBA
1,2,CORNELIO PROCOPIO
2,3,CAMPO MOURAO
3,4,MEDIANEIRA
4,5,PATO BRANCO
5,6,PONTA GROSSA
6,7,DOIS VIZINHOS
7,8,LONDRINA
8,9,TOLEDO
9,10,APUCARANA


In [6]:
q_enrollment_phase_id =\
"""
    SELECT ENROLLMENT_PHASE_ID,
        ENROLLMENT_PHASE_NAME
    FROM ENROLLMENT_PHASE
    ORDER BY ENROLLMENT_PHASE_ID ASC
"""
with psql_connect('../bot/endpoints.yml') as conn:
    enrollment_phase_id = pd.read_sql_query(q_enrollment_phase_id, conn)

In [7]:
enrollment_phase_id

,enrollment_phase_id,enrollment_phase_name
0,1,REQUERIMENTO
1,2,ANALISE DE TURMAS
2,3,AJUSTE
3,4,INCLUSAO
4,5,INTERCAMPUS


### Define URL para scraping, faz uma requisição HTTP e carrega o resultado na memória

In [8]:
url = 'https://portal.utfpr.edu.br/secretaria/matricula/cronograma-de-matricula'
r = requests.get(url)

soup = BeautifulSoup(r.text, 'html.parser')

### Verifica Ano/Semestre da matrícula

In [9]:
title = soup.find('div', class_='documentDescription description').get_text()
year_semester = title.split('-')[-1].strip()
year = int(year_semester.split('/')[0])
semester = int(year_semester.split('/')[1])
print(f'Year: {year}\nSemester: {semester}')

Year: 2022
Semester: 1


### Requerimento

In [17]:
id = int(enrollment_phase_id[enrollment_phase_id['enrollment_phase_name'] == 'REQUERIMENTO']['enrollment_phase_id'].iloc[0])

requerimento = process_table(find_table(soup, 'REQUERIMENTO'), enrollment_phase_id=id, year=year, semester=semester, campus_id=campus_id)
requerimento

,campus_id,enrollment_phase_id,year,semester,start_timestamp,end_timestamp
0,10,1,2022,1,2022-02-14 09:00:00,2022-02-15 18:00:00
1,3,1,2022,1,2022-02-14 09:00:00,2022-02-15 18:00:00
2,2,1,2022,1,2022-02-14 09:00:00,2022-02-15 18:00:00
3,1,1,2022,1,2022-02-16 09:00:00,2022-02-17 18:00:00
4,7,1,2022,1,2022-02-11 09:00:00,2022-02-13 18:00:00
5,11,1,2022,1,2022-02-16 09:00:00,2022-02-18 18:00:00
6,12,1,2022,1,2022-02-14 09:00:00,2022-02-16 18:00:00
7,8,1,2022,1,2022-02-11 09:00:00,2022-02-14 18:00:00
8,4,1,2022,1,2022-02-11 09:00:00,2022-02-14 18:00:00
9,5,1,2022,1,2022-02-12 09:00:00,2022-02-15 18:00:00


### Ajuste/Confirmação

In [18]:
id = int(enrollment_phase_id[enrollment_phase_id['enrollment_phase_name'] == 'AJUSTE']['enrollment_phase_id'].iloc[0])

ajuste = process_table(find_table(soup, 'ajuste[/ ]confirma[cç][aã]o'), enrollment_phase_id=id, year=year, semester=semester, campus_id=campus_id)
ajuste

,campus_id,enrollment_phase_id,year,semester,start_timestamp,end_timestamp
0,10,3,2022,1,2022-02-18 09:00:00,2022-02-18 18:00:00
1,3,3,2022,1,2022-02-18 09:00:00,2022-02-18 18:00:00
2,2,3,2022,1,2022-02-17 09:00:00,2022-02-17 18:00:00
3,1,3,2022,1,2022-02-22 10:00:00,2022-02-22 18:00:00
4,7,3,2022,1,2022-02-17 09:00:00,2022-02-17 18:00:00
5,11,3,2022,1,2022-02-22 09:00:00,2022-02-22 18:00:00
6,12,3,2022,1,2022-02-21 09:00:00,2022-02-21 18:00:00
7,8,3,2022,1,2022-02-17 09:00:00,2022-02-17 18:00:00
8,4,3,2022,1,2022-02-17 09:00:00,2022-02-17 18:00:00
9,5,3,2022,1,2022-02-19 09:00:00,2022-02-21 18:00:00


### Inclusão

In [19]:
id = int(enrollment_phase_id[enrollment_phase_id['enrollment_phase_name'] == 'INCLUSAO']['enrollment_phase_id'].iloc[0])

inclusao = process_table(find_table(soup, 'inclus[aã]o'), enrollment_phase_id=id, year=year, semester=semester, campus_id=campus_id)
inclusao

,campus_id,enrollment_phase_id,year,semester,start_timestamp,end_timestamp
0,10,4,2022,1,2022-02-21 09:00:00,2022-02-21 18:00:00
1,3,4,2022,1,2022-02-21 09:00:00,2022-02-23 18:00:00
2,2,4,2022,1,2022-02-18 09:00:00,2022-02-18 18:00:00
3,1,4,2022,1,2022-02-23 09:00:00,2022-02-23 18:00:00
4,7,4,2022,1,2022-02-18 09:00:00,2022-02-18 18:00:00
5,11,4,2022,1,2022-02-23 09:00:00,2022-02-23 18:00:00
6,12,4,2022,1,2022-02-22 09:00:00,2022-02-22 18:00:00
7,8,4,2022,1,2022-02-18 09:00:00,2022-02-18 18:00:00
8,4,4,2022,1,2022-02-18 09:00:00,2022-02-23 18:00:00
9,5,4,2022,1,2022-02-22 09:00:00,2022-02-23 18:00:00


### Intercampus

In [20]:
id = int(enrollment_phase_id[enrollment_phase_id['enrollment_phase_name'] == 'INTERCAMPUS']['enrollment_phase_id'].iloc[0])

intercampus = process_table(find_table(soup, 'intercampus'), enrollment_phase_id=id, year=year, semester=semester, campus_id=campus_id)
intercampus

,campus_id,enrollment_phase_id,year,semester,start_timestamp,end_timestamp
0,10,5,2022,1,2022-02-24 10:00:00,2022-02-24 18:00:00
1,3,5,2022,1,2022-02-24 10:00:00,2022-02-24 18:00:00
2,2,5,2022,1,2022-02-24 10:00:00,2022-02-24 18:00:00
3,1,5,2022,1,2022-02-24 10:00:00,2022-02-24 18:00:00
4,7,5,2022,1,2022-02-24 10:00:00,2022-02-24 18:00:00
5,11,5,2022,1,2022-02-24 10:00:00,2022-02-24 18:00:00
6,12,5,2022,1,2022-02-24 10:00:00,2022-02-24 18:00:00
7,8,5,2022,1,2022-02-24 10:00:00,2022-02-24 18:00:00
8,4,5,2022,1,2022-02-24 10:00:00,2022-02-24 18:00:00
9,5,5,2022,1,2022-02-24 10:00:00,2022-02-24 18:00:00


### Inserir dados atualizados no banco de dados

In [49]:
enrollment_schedule = pd.concat([requerimento, ajuste, inclusao, intercampus]).sort_values(by=list(enrollment_schedule.columns))
enrollment_schedule

,campus_id,enrollment_phase_id,year,semester,start_timestamp,end_timestamp
3,1,1,2022,1,2022-02-16 09:00:00,2022-02-17 18:00:00
3,1,3,2022,1,2022-02-22 10:00:00,2022-02-22 18:00:00
3,1,4,2022,1,2022-02-23 09:00:00,2022-02-23 18:00:00
3,1,5,2022,1,2022-02-24 10:00:00,2022-02-24 18:00:00
2,2,1,2022,1,2022-02-14 09:00:00,2022-02-15 18:00:00
2,2,3,2022,1,2022-02-17 09:00:00,2022-02-17 18:00:00
2,2,4,2022,1,2022-02-18 09:00:00,2022-02-18 18:00:00
2,2,5,2022,1,2022-02-24 10:00:00,2022-02-24 18:00:00
1,3,1,2022,1,2022-02-14 09:00:00,2022-02-15 18:00:00
1,3,3,2022,1,2022-02-18 09:00:00,2022-02-18 18:00:00


In [50]:
values = ',\n'.join([str(tuple(x)) for x in enrollment_schedule.to_numpy()])
cols = ', '.join(list(enrollment_schedule.columns)).upper()
insert_query =\
f"""
INSERT INTO ENROLLMENT_SCHEDULE ({cols})
VALUES {values}
"""
print(insert_query)


INSERT INTO ENROLLMENT_SCHEDULE (CAMPUS_ID, ENROLLMENT_PHASE_ID, YEAR, SEMESTER, START_TIMESTAMP, END_TIMESTAMP)
VALUES (1, 1, 2022, 1, '2022-02-16 09:00:00', '2022-02-17 18:00:00'),
(1, 3, 2022, 1, '2022-02-22 10:00:00', '2022-02-22 18:00:00'),
(1, 4, 2022, 1, '2022-02-23 09:00:00', '2022-02-23 18:00:00'),
(1, 5, 2022, 1, '2022-02-24 10:00:00', '2022-02-24 18:00:00'),
(2, 1, 2022, 1, '2022-02-14 09:00:00', '2022-02-15 18:00:00'),
(2, 3, 2022, 1, '2022-02-17 09:00:00', '2022-02-17 18:00:00'),
(2, 4, 2022, 1, '2022-02-18 09:00:00', '2022-02-18 18:00:00'),
(2, 5, 2022, 1, '2022-02-24 10:00:00', '2022-02-24 18:00:00'),
(3, 1, 2022, 1, '2022-02-14 09:00:00', '2022-02-15 18:00:00'),
(3, 3, 2022, 1, '2022-02-18 09:00:00', '2022-02-18 18:00:00'),
(3, 4, 2022, 1, '2022-02-21 09:00:00', '2022-02-23 18:00:00'),
(3, 5, 2022, 1, '2022-02-24 10:00:00', '2022-02-24 18:00:00'),
(4, 1, 2022, 1, '2022-02-11 09:00:00', '2022-02-14 18:00:00'),
(4, 3, 2022, 1, '2022-02-17 09:00:00', '2022-02-17 18:00:00'

In [51]:
delete_query =\
"""
DELETE FROM ENROLLMENT_SCHEDULE
"""

In [52]:
with psql_connect('../bot/endpoints.yml') as conn:
    cur = conn.cursor()
    cur.execute(delete_query)
    cur.execute(insert_query)
    cur.close()